In [5]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import unicodedata
import string
import re
import spacy
import nltk
from datasets import load_from_disk
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils import resample
from utils import preprocess
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F
import utils
from features import cpw,spw,wps,cwps,cwr,lwps,lwr,dale_chall,length,check_exact_match,overlap,overlap_syn_fraction,tagme_overlap,bm25_score,word2vec_similarity

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\NeilB\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\NeilB\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ImportError: cannot import name 'overlap' from 'features' (d:\études\pldac\M1S2DAC\RITAL\projet-ri\features.py)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

NameError: name 'torch' is not defined

In [ ]:
import gensim.downloader as api

# Télécharger les embeddings Word2Vec pré-entraînés
word2vec_model = api.load('word2vec-google-news-300')

### Chargement des des données et Prétraitement des données

In [ ]:
wikiqa_data = load_from_disk("wikiqa")
test_data_set = wikiqa_data["test"]
train_data_set = wikiqa_data["train"]
validation_data_set = wikiqa_data["validation"]


def preprocess_examples(examples):
    examples['question'] = [preprocess(q) for q in examples['question']]
    examples['answer'] = [preprocess(a) for a in examples['answer']]
    return examples

train_data_set = train_data_set.map(preprocess_examples, batched=True)
validation_data_set = validation_data_set.map(preprocess_examples, batched=True)
test_data_set = test_data_set.map(preprocess_examples, batched=True)

# Convertir en DataFrame
train_df = pd.DataFrame({
    'question': train_data_set['question'],
    'answer': train_data_set['answer'],
    'label': train_data_set['label']
})

validation_df = pd.DataFrame({
    'question': validation_data_set['question'],
    'answer': validation_data_set['answer'],
    'label': validation_data_set['label']
})

test_df = pd.DataFrame({
    'question': test_data_set['question'],
    'answer': test_data_set['answer'],
    'label': test_data_set['label']
})

In [ ]:
# Sur-échantillonner la classe minoritaire
def balance_classes(df):
    df_majority = df[df.label == 0]
    df_minority = df[df.label == 1]
    
    df_minority_upsampled = resample(df_minority, 
                                     replace=True,     # échantillonner avec remplacement
                                     n_samples=len(df_majority),    # pour faire correspondre la classe majoritaire
                                     random_state=123) # pour la reproductibilité
    
    return pd.concat([df_majority, df_minority_upsampled])

balanced_train_df = balance_classes(train_df)
balanced_validation_df = balance_classes(validation_df)
balanced_test_df = balance_classes(test_df)

# Afficher les statistiques des ensembles de données après suréchantillonnage
print("Ensemble de données d'entraînement :")
print(balanced_train_df['label'].value_counts())
print("\nEnsemble de données de validation :")
print(balanced_validation_df['label'].value_counts())
print("\nEnsemble de données de test :")
print(balanced_test_df['label'].value_counts())

Ensemble de données d'entraînement :
label
0    19320
1    19320
Name: count, dtype: int64

Ensemble de données de validation :
label
0    2593
1    2593
Name: count, dtype: int64

Ensemble de données de test :
label
0    5872
1    5872
Name: count, dtype: int64


In [ ]:
# Réduire la classe majoritaire et suréchantillonner la classe minoritaire
def reduce_and_balance_classes(df):
    df_majority = df[df.label == 0]
    df_minority = df[df.label == 1]
    
    # Réduire la classe majoritaire de 75 %
    df_majority_reduced = resample(df_majority, 
                                   replace=False,    # échantillonner sans remplacement
                                   n_samples=int(len(df_majority) * 0.25),  # 25 % de la classe majoritaire
                                   random_state=123) # pour la reproductibilité
    
    # Suréchantillonner la classe minoritaire pour correspondre à la taille de la classe majoritaire réduite
    df_minority_upsampled = resample(df_minority, 
                                     replace=True,     # échantillonner avec remplacement
                                     n_samples=len(df_majority_reduced),    # pour faire correspondre la classe majoritaire réduite
                                     random_state=123) # pour la reproductibilité
    
    return pd.concat([df_majority_reduced, df_minority_upsampled])

balanced_train_df = reduce_and_balance_classes(train_df)
balanced_validation_df = reduce_and_balance_classes(validation_df)
balanced_test_df = reduce_and_balance_classes(test_df)

# Afficher les statistiques des ensembles de données après réduction et suréchantillonnage
print("Ensemble de données d'entraînement :")
print(balanced_train_df['label'].value_counts())
print("\nEnsemble de données de validation :")
print(balanced_validation_df['label'].value_counts())
print("\nEnsemble de données de test :")
print(balanced_test_df['label'].value_counts())

Ensemble de données d'entraînement :
label
0    4830
1    4830
Name: count, dtype: int64

Ensemble de données de validation :
label
0    648
1    648
Name: count, dtype: int64

Ensemble de données de test :
label
0    1468
1    1468
Name: count, dtype: int64


In [ ]:
balanced_train_df = balanced_train_df.sample(frac=1).reset_index(drop=True)
balanced_validation_df = balanced_validation_df.sample(frac=1).reset_index(drop=True)
balanced_test_df = balanced_test_df.sample(frac=1).reset_index(drop=True)

In [ ]:
# Fonction pour obtenir le vecteur moyen de Word2Vec pour un texte donné
def get_mean_word2vec(text, model):
    words = text.split()
    word_vecs = [model[word] for word in words if word in model]
    if len(word_vecs) == 0:
        return np.zeros(model.vector_size)
    return np.mean(word_vecs, axis=0)

# Transformer les questions en vecteurs Word2Vec moyens
def transform_questions(df, model):
    return np.vstack(df['question'].apply(lambda x: get_mean_word2vec(x, model)).values)

# Transformer les réponses en vecteurs Word2Vec moyens
def transform_answers(df, model):
    return np.vstack(df['answer'].apply(lambda x: get_mean_word2vec(x, model)).values)

# Ajouter les nouvelles fonctionnalités pour les questions et les réponses
def transform_length_answer(df):
    return np.array(df['answer'].apply(length).values).reshape(-1, 1)

def transform_exact_match(df):
    return np.array(df.apply(lambda row: check_exact_match(row['question'], row['answer']), axis=1)).reshape(-1, 1)

def transform_overlap(df):
    return np.array(df.apply(lambda row: overlap(row['question'], row['answer']), axis=1)).reshape(-1, 1)

def transform_overlap_syn(df):
    return np.array(df.apply(lambda row: overlap_syn_fraction(row['question'], row['answer']), axis=1)).reshape(-1, 1)

def transform_tagme(df):
    return np.array(df.apply(lambda row: tagme_overlap(row['question'], row['answer']), axis=1)).reshape(-1, 1)

def transform_w2v_sim(df, model):
    return np.array(df.apply(lambda row: word2vec_similarity(row['question'], row['answer'], model), axis=1)).reshape(-1, 1)

# Transformer les questions en vecteurs Word2Vec moyens
def transform_questions(df, model):
    return np.vstack(df['question'].apply(lambda x: get_mean_word2vec(x, model)).values)

# Transformer les réponses en vecteurs Word2Vec moyens
def transform_answers(df, model):
    return np.vstack(df['answer'].apply(lambda x: get_mean_word2vec(x, model)).values)

In [8]:
# Préparer les données pour PyTorch
class QADataset(Dataset):
    def __init__(self, df, word2vec_model):
        self.labels = df['label'].values
        self.scaler = StandardScaler()

        self.features = np.hstack([
            transform_questions(df, word2vec_model),
            transform_answers(df, word2vec_model),
            transform_length_answer(df),
            transform_exact_match(df),
            transform_overlap(df),
            transform_overlap_syn(df),
            transform_tagme(df),
            transform_w2v_sim(df, word2vec_model),

        ])

        self.features = self.scaler.fit_transform(self.features)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return torch.tensor(self.features[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)

# Créer les datasets
train_dataset = QADataset(balanced_train_df, word2vec_model)
validation_dataset = QADataset(balanced_validation_df, word2vec_model)
test_dataset = QADataset(balanced_test_df, word2vec_model)

# Créer les DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# class QACNN(nn.Module):
#     def __init__(self, input_dim):
#         super(QACNN, self).__init__()
#         # Convolution Layer 1
#         self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=3, padding=1)
        
#         # Convolution Layer 2
#         self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        
#         # Max Pooling Layer
#         self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        
#         # Fully Connected Layer 1
#         self.fc1 = nn.Linear(128 * (input_dim // 2 // 2), 128)
        
#         # Fully Connected Layer 2 (Output Layer)
#         self.fc2 = nn.Linear(128, 2)
        
#         # Dropout Layer
#         self.dropout = nn.Dropout(0.5)
    
#     def forward(self, x):
#         # Add a channel dimension
#         x = x.unsqueeze(1)
        
#         # Apply Convolution Layer 1 and ReLU activation
#         x = self.pool(F.relu(self.conv1(x)))
        
#         # Apply Convolution Layer 2 and ReLU activation
#         x = self.pool(F.relu(self.conv2(x)))
        
#         # Flatten the output
#         x = x.view(x.size(0), -1)
        
#         # Apply Fully Connected Layer 1 and ReLU activation
#         x = F.relu(self.fc1(x))
        
#         # Apply Dropout
#         x = self.dropout(x)
        
#         # Apply Fully Connected Layer 2 (Output Layer)
#         x = self.fc2(x)
        
#         return x

# input_dim = train_dataset.features.shape[1]
# model = QACNN(input_dim)

In [32]:
class QACNN(nn.Module):
    def __init__(self, input_dim):
        super(QACNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(64)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(128)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(256)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(256 * (input_dim // 8), 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 2)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = x.unsqueeze(1)  # Ajouter une dimension channel
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = x.view(x.size(0), -1)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

input_dim = train_dataset.features.shape[1]
model = QACNN(input_dim)


### Entraînement du données

In [33]:
# Définir la fonction de perte et l'optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entraîner le modèle
num_epochs = 30
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")



Epoch [1/20], Loss: 0.8069
Epoch [2/20], Loss: 0.6887
Epoch [3/20], Loss: 0.6697
Epoch [4/20], Loss: 0.6396
Epoch [5/20], Loss: 0.6048
Epoch [6/20], Loss: 0.5801
Epoch [7/20], Loss: 0.5405
Epoch [8/20], Loss: 0.4943
Epoch [9/20], Loss: 0.4597
Epoch [10/20], Loss: 0.4344
Epoch [11/20], Loss: 0.4127
Epoch [12/20], Loss: 0.3920
Epoch [13/20], Loss: 0.3625
Epoch [14/20], Loss: 0.3564
Epoch [15/20], Loss: 0.3441
Epoch [16/20], Loss: 0.3356
Epoch [17/20], Loss: 0.3427
Epoch [18/20], Loss: 0.3288
Epoch [19/20], Loss: 0.3134
Epoch [20/20], Loss: 0.3155


### Evaluation sur les données de validation

In [38]:
# Évaluer le modèle sur les données de validation
model.eval()
all_predictions = []
all_labels = []
all_probabilities = []

with torch.no_grad():
    for inputs, labels in validation_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        probabilities = torch.softmax(outputs, dim=1)
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_probabilities.extend(probabilities.cpu().numpy())

validation_probabilities = np.array(all_probabilities)
validation_labels = np.array(all_labels)

print("\nValidation Classification Report:")
print(classification_report(validation_labels, all_predictions))



Validation Classification Report:
              precision    recall  f1-score   support

           0       0.59      0.90      0.71       648
           1       0.79      0.38      0.52       648

    accuracy                           0.64      1296
   macro avg       0.69      0.64      0.62      1296
weighted avg       0.69      0.64      0.62      1296



In [39]:
import numpy as np

# Calculer les métriques de validation
map_score, mrr_score, success_at_1_score = utils.compute_metrics(balanced_validation_df, all_predictions, validation_probabilities)
print(f"\nValidation Results:\nMAP: {map_score}\nMRR: {mrr_score}\nS@1: {success_at_1_score}\n")


Validation Results:
MAP: 0.40922568669006865
MRR: 0.38425925925925924
S@1: 0.3296296296296296



### Evaluation sur les données de test

In [40]:
# Évaluer le modèle sur les données de test
all_predictions = []
all_labels = []
all_probabilities = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        probabilities = torch.softmax(outputs, dim=1)
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_probabilities.extend(probabilities.cpu().numpy())

test_probabilities = np.array(all_probabilities)
test_labels = np.array(all_labels)

print("\nTest Classification Report:")
print(classification_report(test_labels, all_predictions))


Test Classification Report:
              precision    recall  f1-score   support

           0       0.58      0.88      0.70      1468
           1       0.76      0.37      0.49      1468

    accuracy                           0.62      2936
   macro avg       0.67      0.62      0.60      2936
weighted avg       0.67      0.62      0.60      2936



In [41]:
# Calculer les métriques de test
map_score, mrr_score, success_at_1_score = utils.compute_metrics(balanced_test_df, all_predictions, test_probabilities)
print(f"\nTest Results:\nMAP: {map_score}\nMRR: {mrr_score}\nS@1: {success_at_1_score}\n")


Test Results:
MAP: 0.37046201472272033
MRR: 0.34556542406805113
S@1: 0.29071803852889666

